# Eat Safe, Love

## Notebook Set Up

In [147]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [148]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [149]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [150]:
# review the collections in our database
db.list_collection_names()

['establishments']

In [151]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [152]:
# Find the establishments with a hygiene score of 20
hygiene_query = {'scores.Hygiene': 20}
fields = {'BusinessName': 1, 'scores.Hygiene': 1}

results=establishments.find(hygiene_query, fields)

# Use hygiene_count to display the total no. of establishemnts with score of 20.
hygiene_results = establishments.find(hygiene_query, fields)
hygiene_count = establishments.count_documents(hygiene_query)

#Print results 
print(f"Number of documents with hygiene score 20: {hygiene_count}\n")
pprint(hygiene_results[0])

Number of documents with hygiene score 20: 41

{'BusinessName': 'The Chase Rest Home',
 '_id': ObjectId('66c67670051c039cde4b10c1'),
 'scores': {'Hygiene': 20}}


In [153]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
print(f"Total rows in DataFrame: {len(hygiene_df)}")

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)


Total rows in DataFrame: 41


,_id,BusinessName,scores
0,66c67670051c039cde4b10c1,The Chase Rest Home,{'Hygiene': 20}
1,66c67670051c039cde4b1442,Brenalwood,{'Hygiene': 20}
2,66c67670051c039cde4b174d,Melrose Hotel,{'Hygiene': 20}
3,66c67670051c039cde4b193b,Seaford Pizza,{'Hygiene': 20}
4,66c67670051c039cde4b1949,Golden Palace,{'Hygiene': 20}
5,66c67670051c039cde4b22e1,Ashby's Butchers,{'Hygiene': 20}
6,66c67670051c039cde4b2506,South Sea Express Cuisine,{'Hygiene': 20}
7,66c67670051c039cde4b3a34,Golden Palace,{'Hygiene': 20}
8,66c67670051c039cde4b3e78,The Tulip Tree,{'Hygiene': 20}
9,66c67670051c039cde4b468b,F & S,{'Hygiene': 20}


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [161]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
london_query = {
    'LocalAuthorityName': {'$regex': 'London'},
    'RatingValue': {'$gte': 4}
}
london_fields = {'BusinessName': 1, 'LocalAuthorityName': 1, 'RatingValue': 1}

# Use count_documents to display the number of documents in the result
london_results = establishments.find(london_query, london_fields)
london_count = establishments.count_documents(london_query)

# Display the first document in the results using pprint
print(f"\nNumber of documents with RatingValue >= 4 in London: {london_count}\n")
pprint(london_results[0])


Number of documents with RatingValue >= 4 in London: 33

{'BusinessName': "Charlie's",
 'LocalAuthorityName': 'City of London Corporation',
 'RatingValue': 4,
 '_id': ObjectId('66c67670051c039cde4b2ad9')}


In [162]:
# Convert the result to a Pandas DataFrame
london_df = pd.DataFrame(list(london_results))
# Display the number of rows in the DataFrame
print(f"Total rows in DataFrame: {len(london_df)}")
# Display the first 10 rows of the DataFrame
london_df.head(10)

Total rows in DataFrame: 33


,_id,BusinessName,RatingValue,LocalAuthorityName
0,66c67670051c039cde4b2ad9,Charlie's,4,City of London Corporation
1,66c67670051c039cde4b2e02,Mv City Cruises Erasmus,5,City of London Corporation
2,66c67670051c039cde4b394f,Benfleet Motor Yacht Club,4,City of London Corporation
3,66c67670051c039cde4b474c,Coombs Catering t/a The Lock and Key,5,City of London Corporation
4,66c67670051c039cde4b474d,Tilbury Seafarers Centre,5,City of London Corporation
5,66c67670051c039cde4b4e58,Mv Valulla,5,City of London Corporation
6,66c67670051c039cde4b6f62,Tereza Joanne,5,City of London Corporation
7,66c67670051c039cde4b7330,Brick Lane Brews,4,City of London Corporation
8,66c67670051c039cde4b7347,The Nuance Group (UK) Limited,5,City of London Corporation
9,66c67670051c039cde4b7348,WH Smith,5,City of London Corporation


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [184]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
penang_query = {'BusinessName': 'Penang Flavours'}
penang_fields = {
    'BusinessName': 1, 
    'geocode.latitude': 1, 
    'geocode.longitude': 1,
    'RatingValue': 1,  # Include RatingValue in the fields
    'scores.Hygiene': 1  # Include Hygiene scores in the fields
}
penang_location = establishments.find_one(penang_query, penang_fields)

# Define the search range and query for nearby establishments
degree_margin = 0.01
latitude = penang_location['geocode']['latitude']
longitude = penang_location['geocode']['longitude']

top5_query = {
    'RatingValue': 5,
    'geocode.latitude': {'$gte': latitude - degree_margin, '$lte': latitude + degree_margin},
    'geocode.longitude': {'$gte': longitude - degree_margin, '$lte': longitude + degree_margin},
    'scores.Hygiene': {'$ne': None}
}
top5_sort = [('scores.Hygiene', 1)]

# Fetch and sort the top 5 results
top5_results = establishments.find(top5_query, penang_fields).sort(top5_sort).limit(5)

# Collect data for DataFrame conversion
top5_data = []
for establishment in top5_results:
    top5_data.append({
        'BusinessName': establishment.get('BusinessName', 'N/A'),
        'RatingValue': establishment.get('RatingValue', 'N/A'),
        '_id': establishment.get('_id', 'N/A'),
        'latitude': establishment.get('geocode', {}).get('latitude', 'N/A'),
        'longitude': establishment.get('geocode', {}).get('longitude', 'N/A'),
        'Hygiene': establishment.get('scores', {}).get('Hygiene', 'N/A')
    })

top5_df = pd.DataFrame(top5_data)
#pprint(top5_df)
top5_df.head(5)


,BusinessName,RatingValue,_id,latitude,longitude,Hygiene
0,Volunteer,5,66c67670051c039cde4b691e,51.487344,0.092080,0
1,Lumbini Grocery Ltd T/A Al-Iman,5,66c67670051c039cde4b6938,51.487163,0.091626,0
2,Plumstead Manor Nursery,5,66c67670051c039cde4b6939,51.481518,0.085994,0
3,Iceland,5,66c67670051c039cde4b68f3,51.487148,0.092420,0
4,Howe and Co Fish and Chips - Van 17,5,66c67670051c039cde4b6904,51.487534,0.092537,0


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [185]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
        {"$match": {"scores.Hygiene": 0}},
        {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
         {"$sort": {"count": -1}}
         
         ]
# Print the number of documents in the result
results = list(establishments.aggregate(pipeline))
# Print the first 10 results
pprint(results[0:10])

[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [159]:
# Convert the result to a Pandas DataFrame
result_df= pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Number of rows is ", len(result_df))
# Display the first 10 rows of the DataFrame
result_df.head(10)

Number of rows is  55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
